In [1]:
import os
import shutil

# Original dataset path (38 classes)
original_dataset = "Crop Disease Detection/dataset/plantvillage dataset/color"  # <-- update if different
binary_dataset = "Crop Disease Detection/dataset_binary"  # Output folder

# Binary folder names
healthy_classes = [
    'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___healthy',
    'Corn_(maize)___healthy', 'Grape___healthy', 'Peach___healthy',
    'Pepper,_bell___healthy', 'Potato___healthy', 'Raspberry___healthy',
    'Soybean___healthy', 'Strawberry___healthy', 'Tomato___healthy'
]

# Prepare folders
healthy_folder = os.path.join(binary_dataset, 'Healthy')
unhealthy_folder = os.path.join(binary_dataset, 'Unhealthy')
os.makedirs(healthy_folder, exist_ok=True)
os.makedirs(unhealthy_folder, exist_ok=True)

# Move images
for class_folder in os.listdir(original_dataset):
    class_path = os.path.join(original_dataset, class_folder)
    if not os.path.isdir(class_path):
        continue

    if class_folder in healthy_classes:
        dest_folder = healthy_folder
    else:
        dest_folder = unhealthy_folder

    for img in os.listdir(class_path):
        src = os.path.join(class_path, img)
        dst = os.path.join(dest_folder, f"{class_folder}_{img}")
        shutil.copy2(src, dst)

print("✅ Dataset organized into binary classification:")
print(f" - Healthy: {len(os.listdir(healthy_folder))} images")
print(f" - Unhealthy: {len(os.listdir(unhealthy_folder))} images")


✅ Dataset organized into binary classification:
 - Healthy: 15084 images
 - Unhealthy: 39221 images


In [4]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths
dataset_dir = "Crop Disease Detection/dataset_binary"  # <-- updated for binary dataset
model_dir = "Crop Disease Detection/model"
os.makedirs(model_dir, exist_ok=True)

# Image dimensions
img_height, img_width = 128, 128
batch_size = 32

# Data Generator
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_data = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',  # <-- for binary classification
    subset='training',
    shuffle=True
)

val_data = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',  # <-- for binary classification
    subset='validation'
)

# Save class indices to use during prediction
class_indices = train_data.class_indices
with open(os.path.join(model_dir, "class_indices.json"), "w") as f:
    json.dump(class_indices, f)

# CNN Model for Binary Classification
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # <-- for binary output
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',  # <-- updated for binary
              metrics=['accuracy'])

model.summary()

# Train
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

# Save the model
model_path = os.path.join(model_dir, "crop_disease_model.keras")
model.save(model_path)
print(f"✅ Model saved to: {model_path}")


Found 43445 images belonging to 2 classes.
Found 10860 images belonging to 2 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856     
                                    

In [ ]:
import os
import json
import numpy as np
from flask import Flask, render_template, request
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load model and class labels
model = load_model('Crop Disease Detection/model/crop_disease_model.keras')

with open('Crop Disease Detection/model/class_indices.json') as f:
    class_indices = json.load(f)

# Reverse the class indices
class_labels = {v: k for k, v in class_indices.items()}

# Flask app setup
app = Flask(__name__, template_folder='Crop Disease Detection/templates')
UPLOAD_FOLDER = 'Crop Disease Detection/static/upload'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return render_template('index.html', error='No file part')
    
    file = request.files['file']
    if file.filename == '':
        return render_template('index.html', error='No selected file')

    if file:
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        file.save(file_path)

        # Preprocess the image
        img = image.load_img(file_path, target_size=(128, 128))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0

        # Prediction
        prediction = model.predict(img_array)[0][0]
        predicted_class = "Unhealthy" if prediction >= 0.5 else "Healthy"
        confidence = prediction if predicted_class == "Unhealthy" else 1 - prediction

        # Static info
        info = {
            "Healthy": {
                "description": "The crop appears healthy with no visible disease symptoms.",
                "precautions": "Continue regular monitoring and proper care."
            },
            "Unhealthy": {
                "description": "The crop might be infected by a disease. Further diagnosis recommended.",
                "precautions": "Isolate affected crops, consult an expert, and apply proper treatment."
            }
        }

        return render_template('index.html',
                               prediction=predicted_class,
                               description=info[predicted_class]["description"],
                               precautions=info[predicted_class]["precautions"],
                               confidence=f"{confidence*100:.2f}%",
                               image_path=file_path)

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)

    
    

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [25/Apr/2025 16:56:19] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 0s 149ms/step


127.0.0.1 - - [25/Apr/2025 16:56:27] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 34ms/step


127.0.0.1 - - [25/Apr/2025 16:56:38] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 33ms/step


127.0.0.1 - - [25/Apr/2025 16:56:47] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 30ms/step


127.0.0.1 - - [25/Apr/2025 16:56:55] "POST /predict HTTP/1.1" 200 -
